In [19]:
var csv = require('csv-parse')
var parse = require('csv-parse/lib/sync')
var fs = require('fs')
var axios = require('axios')


var raw = fs.readFileSync(__dirname + '/2015senate.csv')
raw = parse(raw, {delimiter: ','})




[ [ '',
    '',
    '',
    '',
    '',
    '189th ',
    '',
    '',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189th SENATE',
    '189S',
    '189S',
    '189S',
    '189S',
    '189s',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189th SENATE',
    '189S',
    '189S',
    '189S',
    '189S',
    '188s',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189th SENATE',
    '189S',
    '189S',
    '189s',
    '189S',
    '189S',
    '189S',
    '189S',
    '189S',
    '189th SENATE',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '' ],
  [ '',
    '',
    '',
    '',
    '',
    'SENATE',
    '',
    '',
    '1s',
    '2s',
    '3s',
    '4s',
    '5s',
    '6s',
    '7s',
    '8s',
    '9s',
    '10s',
    'Vote Key ',
    '11s',
    '12s',
    '13s',
    '14s',
    '15s',
    '16s',
    '17s',
    '18s',
    '19s',
    '20s'

In [20]:
// some cells are useless, get rid of them
function filterVoteItems (row){
    return row.filter((r, i)=>{
        return ([10, 21, 32].indexOf(i) < 0)
    })
}

undefined

In [21]:
progressivePosition = filterVoteItems(raw[2].slice(8, 49))
bills = filterVoteItems(raw[3].slice(8, 49))
rolecall = filterVoteItems(raw[4].slice(8, 49))


[ '21',
  '23',
  '24',
  '44',
  '45',
  '49',
  '90',
  '160',
  '161',
  '164',
  '190',
  '211',
  '212',
  '232',
  '269',
  '272',
  '296',
  '297',
  '298',
  '303',
  '326',
  '328',
  '331',
  '333',
  '334',
  '363',
  '364',
  '374',
  '400',
  '401',
  '406',
  '407',
  '412',
  '413',
  '418',
  '420',
  '422',
  '537' ]

In [22]:
var senatorData = {};

raw.slice(5)
// ignore metadata rows
.filter(function(l){
    return (l[1] % 1 === 0)
})
.filter((l)=>{
    //l[5] = name
    return l[5] !== 'z special election space'
})
.forEach(function(l){
    const votes = filterVoteItems(l.slice(8, 49))
    .map((v,i)=>{
        let billName = bills[i]
        // some of the bill names are a little off
        billName = billName
            .replace(/\s/g, '')
            .replace(/S(?=\d)/, 'S.')
            .replace(/H(?=\d)/, 'H.')

        return {
       [billName + ' ' + rolecall[i]] : v.trim()
        }
    })
    
    const name = l[5].trim()
    const district = l[57].trim()
        
    if (!name) return
        
     const lastName = name.split(',')[0].split(' ')[0]
     const firstName = name.split(',')[1].trim().split(' ')[0]
    
     console.log(Object.values(votes[votes.length -1])[0])
    if (Object.values(votes[votes.length -1])[0] == 'n/a'){
        // its someone who dropped out midterm, dont put
        // them in the objet, they'll override the more recent person
        console.log(name)
        return
    }
     
    senatorData[district] = {
        '2015-2016' : {
            senator : firstName + ' ' + lastName,
            votes : votes,
            voteRating : l[4]
        },
        '2017-2018' : {
            name : undefined
        },
        senators : {
            [firstName + ' ' + lastName] : {
                phone : l[52],
                email : l[53],
                twitter : l[54],
                party: l[6].match(/\((\w)\)/) ? l[6].match(/\((\w)\)/)[1] : l[6]
            }
        }    
    }
    
    axios.get('http://openstates.org/api/v1/legislators/', {
        params : {
              state : 'ma',
              last_name : lastName,
//               first_name : firstName,
              active : 'false',
              chamber: 'upper',
          }  
})
  .then(function (response) {
        
  if (response.data.length === 0){
      console.log(lastName, firstName), 'couldnt find data'
      return
  }
    senatorData[district].senators[firstName + ' ' + lastName].photo_url = response.data[0].photo_url
    senatorData[district].senators[firstName + ' ' + lastName].url = response.data[0].url        
  })

})



+
+
+
+
+
+
+
-
+
+
+
+
+
-
+
+
-
n/a
Hedlund, Robert
-
+
+
+
n/a
Kennedy, Thomas *
+
+
+
+
+
+
-
-
+
+
n/a
Petruccelli, Anthony
-
+
-
+
+
-
-
+
+


undefined

Wolf Daniel
Downing Benjamin
Joyce Brian


In [23]:
  simplifyDistrict = (district) => {
      return district
      .replace(/\band\b/g, '')
      .replace(/\W/g, '')
      .replace(/District/g, '')
    }
 
  
axios.get('http://openstates.org/api/v1/legislators/?state=ma&chamber=upper')
  .then(function (response) { 
        const data = response.data
        data.forEach((d)=>{
            if (!senatorData[d.district]){
                console.log(d.district)
                return
            }
            const full_name_without_middle = d.full_name.replace(/\s\w\.\s/, ' ')
            senatorData[d.district]['2017-2018'].senator = full_name_without_middle
            if (full_name_without_middle !==  senatorData[d.district]['2015-2016'].senator){
                 senatorData[d.district].senators[full_name_without_middle] = {
                    url : d.url,
                    photo_url : d.photo_url,
                    party : d.party
                 }
            }

        })
     }) 



Promise { <pending> }

In [24]:
senatorData

{ 'Third Middlesex': 
   { '2015-2016': 
      { senator: 'Michael Barrett',
        votes: [Object],
        voteRating: '87%' },
     '2017-2018': { name: 'Michael Barrett' },
     senators: { 'Michael Barrett': [Object] } },
  'First Suffolk and Middlesex': 
   { '2015-2016': { senator: 'Joseph Boncore', votes: [Object], voteRating: '90%' },
     '2017-2018': { name: 'Joseph Boncore' },
     senators: { 'Joseph Boncore': [Object] } },
  'Second Plymouth and Bristol': 
   { '2015-2016': { senator: 'Michael Brady', votes: [Object], voteRating: '85%' },
     '2017-2018': { name: 'Michael Brady' },
     senators: { 'Michael Brady': [Object] } },
  'Second Suffolk and Middlesex': 
   { '2015-2016': 
      { senator: 'William Brownsberger',
        votes: [Object],
        voteRating: '79%' },
     '2017-2018': { name: 'William Brownsberger' },
     senators: { 'William Brownsberger': [Object] } },
  'First Worcester': 
   { '2015-2016': 
      { senator: 'Harriette Chandler',
        vot

In [25]:
fs.writeFile('./senate_votes.json', JSON.stringify(senatorData), function(err) {
  if (err) {
    return console.log(err);
  }

  console.log("The file was saved!");
});

undefined

The file was saved!


In [26]:
axios.get('http://openstates.org/api/v1/legislators/', {
        params : {
              state : 'ma',
              last_name : 'Petruccelli',
              active : false
          }  
})
  .then(function (response) {
    console.log(response.data);
  })
  .catch(function (error) {
    console.log(error);
  });

Promise { <pending> }

[ { last_name: 'Petruccelli',
    updated_at: '2016-02-19 19:34:05',
    full_name: 'Anthony Petruccelli',
    id: 'MAL000030',
    first_name: 'Anthony W.',
    middle_name: '',
    state: 'ma',
    votesmart_id: '1965',
    email: 'Anthony.Petruccelli@masenate.gov',
    all_ids: [ 'MAL000030', 'MAL000265', 'MAL000288' ],
    leg_id: 'MAL000030',
    active: false,
    transparencydata_id: 'c0fec2fae18340f39b4d380dde5dc937',
    photo_url: 'https://malegislature.gov/Images/MemberProfile/Web/apetruccelli.jpg',
    level: 'state',
    url: 'http://www.malegislature.gov/People/Profile/AWP0',
    country: 'us',
    created_at: '2011-07-08 21:34:37',
    offices: [ [Object] ],
    suffixes: '' } ]
